In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.io as sio
%matplotlib inline

# GPU settings
config = tf.ConfigProto()
config.allow_soft_placement = True
config.gpu_options.per_process_gpu_memory_fraction = 0.4
config.gpu_options.allow_growth = True

C:\Users\dongwan123\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load data
mat = sio.loadmat('./train_data/trainData_i80_0400-0415.mat')
f_data = np.array(mat['f_data'])
prevTraj_data = np.array(mat['prevTraj_data'])
postTraj_data = np.array(mat['postTraj_data'])

# Set train-data
f_trainData = f_data
prevTraj_trainData = prevTraj_data
postTraj_trainData = postTraj_data

num_traindata = f_trainData.shape[0] # number of train-data

f_mean_trainData = np.mean(f_trainData,axis=0)
prevTraj_mean_trainData = np.mean(prevTraj_trainData,axis=0)
postTraj_mean_trainData = np.mean(postTraj_trainData,axis=0)

f_trainData = f_trainData - np.tile(f_mean_trainData,(num_traindata,1))
prevTraj_trainData = prevTraj_trainData - np.tile(prevTraj_mean_trainData,(num_traindata,1))
postTraj_trainData = postTraj_trainData - np.tile(postTraj_mean_trainData,(num_traindata,1))

f_absmax = np.max(np.abs(f_trainData),axis=0)
f_trainData = f_trainData/np.tile(f_absmax,(num_traindata,1))
prevTraj_absmax = np.max(np.abs(prevTraj_trainData),axis=0)
prevTraj_trainData = prevTraj_trainData/np.tile(prevTraj_absmax,(num_traindata,1))
postTraj_absmax = np.max(np.abs(postTraj_trainData),axis=0)
postTraj_trainData = postTraj_trainData/np.tile(postTraj_absmax,(num_traindata,1))

dim_x = 3               # dimension of state
Hx_prev = 15            # timesteps of input (horizon) : less than 6
Hx_post = 15            # timesteps of post (horizon) : less than 10
dim_t = dim_x*Hx_post   # dimension of trajectory
dim_f = f_trainData.shape[1]

OSError: could not read bytes

In [3]:
# Network Parameters
n_hidden_lstm = dim_x*3
n_components_mdn = 12
n_outputs_mdn = (2*dim_t + 1)*n_components_mdn
lam_mdn = 0.01

#hidden_size_mdn = n_outputs_mdn*2
hidden_size_mdn = [(n_outputs_mdn)*4, (n_outputs_mdn)*4]

sigma_max = 10 # max value of variance

# Set epsilon value
epsilon_init = 1e-5
decayRate_epsilon =0.95
decaySteps_epsilon = 200
learning_epsilon = epsilon_init

# tf Graph input
X = tf.placeholder("float", [None, Hx_prev, dim_x])
Y = tf.placeholder("float", [None, dim_t])
F = tf.placeholder("float", [None, dim_f])
EPS = tf.placeholder("float", None)

# Save params
filename2save0 = "./tf_saved_model/params_predTraj_lstmMDN_ti%dto%d" % (Hx_prev,Hx_post)
sio.savemat(filename2save0, {'f_mean_trainData':f_mean_trainData,'prevTraj_mean_trainData':prevTraj_mean_trainData,
                             'postTraj_mean_trainData':postTraj_mean_trainData,'f_absmax':f_absmax,'prevTraj_absmax':prevTraj_absmax,
                             'postTraj_absmax':postTraj_absmax,'n_hidden_lstm':n_hidden_lstm,'n_components_mdn':n_components_mdn,
                             'sigma_max':sigma_max})

In [4]:
def LSTMwMDN(x_, f_):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    
    x_ = tf.unstack(x_, Hx_prev, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden_lstm)

    # Get lstm cell output
    rnn_outputs, rnn_states = rnn.static_rnn(lstm_cell, x_, dtype=tf.float32)
    
    rnn_encoded = rnn_outputs[-1]
    
    x_concat = tf.concat([rnn_encoded, f_], 1)

    output_mdn_1 = tf.layers.dense(x_concat, 
                           hidden_size_mdn[0], 
                           activation = tf.nn.relu, 
                           kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                           bias_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                           kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lam_mdn)
                          )
    
    output_mdn_2 = tf.layers.dense(output_mdn_1, 
                           hidden_size_mdn[1], 
                           activation = tf.nn.relu,
                           kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                           bias_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                           kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lam_mdn)
                          )
    output_mdn_out = tf.layers.dense(output_mdn_2, 
                           n_outputs_mdn,
                           kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                           bias_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                           kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lam_mdn)
                          )
    
    indexes_split = [dim_t*n_components_mdn, dim_t*n_components_mdn, n_components_mdn]
    means_mdn_out, sigma_mdn_act, fracs_mdn_act = tf.split(output_mdn_out, indexes_split, axis=1)

    sigma_mdn_out = sigma_max*tf.nn.sigmoid(sigma_mdn_act)

    fracs_mdn_out_max = tf.reduce_max(fracs_mdn_act,axis=1)
    fracs_mdn_out_max_ext_ = tf.expand_dims(fracs_mdn_out_max, 1)
    fracs_mdn_out_max_ext = tf.tile(fracs_mdn_out_max_ext_,[1, n_components_mdn])
    fracs_mdn_out = fracs_mdn_act - fracs_mdn_out_max_ext
    fracs_mdn_out = tf.nn.softmax(fracs_mdn_out)
    
    return means_mdn_out, sigma_mdn_out, fracs_mdn_out

In [5]:
def logsumexp_trick(x_):
    xmax_ = tf.reduce_max(x_,axis = 1)
    xmax_ext__ = tf.expand_dims(xmax_, 1)
    xmax_ext_ = tf.tile(xmax_ext__,[1, n_components_mdn])
    diffx_ = x_ - xmax_ext_
    x_logsumexp_ = tf.reduce_logsumexp(diffx_,1)
    x_logsumexp = xmax_ + x_logsumexp_
    
    return x_logsumexp

In [6]:
# RNN graph + MDN
means_mdn_, sigma_mdn_, fracs_mdn_ = LSTMwMDN(X, F)
means_mdn = tf.reshape(means_mdn_,[-1,dim_t,n_components_mdn]) # (num of data, dim_out*horizon_out, num of components)
sigma_mdn = tf.reshape(sigma_mdn_,[-1,dim_t,n_components_mdn]) # (num of data, dim_out*horizon_out, num of components)
fracs_mdn = tf.reshape(fracs_mdn_,[-1,n_components_mdn]) # (num of data, num of components)

# Define loss and optimizer
Y_ext_ = tf.expand_dims(Y, 2)
Y_ext = tf.tile(Y_ext_,[1,1,n_components_mdn]) # (num of data, dim_out*horizon_out, num of components)
diff = Y_ext - means_mdn

squared_diff = tf.square(diff)
scaled_squared_diff = tf.div(squared_diff,(sigma_mdn+EPS))
scaled_dist = tf.reduce_sum(scaled_squared_diff,1) # (num of data, num of components)

logsum_sigma_mdn = tf.reduce_sum(tf.log(sigma_mdn+EPS),1) # (num of data, num of components)

loss_exponent = tf.log(fracs_mdn+EPS) - 0.5*dim_t*tf.log(np.pi*2) - 0.5*logsum_sigma_mdn - 0.5*scaled_dist
# loss_op = tf.reduce_mean(-tf.reduce_logsumexp(loss_exponent,axis=1))
loss_op = -tf.reduce_mean(logsumexp_trick(loss_exponent))

learning_rate = 0.001
# train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss_op)
train_op = tf.train.RMSPropOptimizer(learning_rate=learning_rate, decay=0.8).minimize(loss_op)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Create a saver object which will save all the variables
saver = tf.train.Saver()

# Export meta graph
tf.add_to_collection('X', X)
tf.add_to_collection('Y', Y)
tf.add_to_collection('F', F)
tf.add_to_collection('means_mdn', means_mdn)
tf.add_to_collection('sigma_mdn', sigma_mdn)
tf.add_to_collection('fracs_mdn', fracs_mdn)

In [7]:
# Training parameters
training_epochs = 1000
batch_size = 256
num_batches = np.ceil(num_traindata/batch_size)
num_batches = num_batches.astype(np.int32)
display_epochs = 50
save_epochs = 200

h_x_train = np.array(range(0,dim_x*Hx_prev), dtype = np.int32)
h_y_train = np.array(range(0,dim_x*Hx_post), dtype = np.int32)

# Suffle indexes
idx_array = np.arange(num_traindata)
np.random.shuffle(idx_array)

with tf.Session(config = config) as sess:
    # Run the initializer
    sess.run(init)
    
    loss_epochs = np.zeros(training_epochs)
    # Train phase
    for epoch in range(1, training_epochs+1):
        
        # Suffle indexes
        np.random.shuffle(idx_array)
        
        # Set epsilon value
        if (epoch % decaySteps_epsilon) == 0 and epoch > 1:
            learning_epsilon = learning_epsilon*decayRate_epsilon
        
        cnt_data = 0
        loss_array = np.zeros(num_batches)
        for nidx_batch in range(0,num_batches):
            # Get batch data
            if((cnt_data + batch_size) > num_traindata):
                idx_batch = idx_array[range((cnt_data),(num_traindata))]
            else:
                idx_batch = idx_array[range((cnt_data),(cnt_data + batch_size))]
                
            x_batch = prevTraj_trainData[idx_batch,:]
            x_batch = x_batch[:,h_x_train]
            x_batch = x_batch.reshape(len(idx_batch),Hx_prev,dim_x)  # convert to (batch_size, timesteps, dim_x)
            
            y_batch = postTraj_trainData[idx_batch,:]
            y_batch = y_batch[:,h_y_train]
            
            f_batch = f_trainData[idx_batch,:]
            
            cnt_data = cnt_data + len(idx_batch)
            
            # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: x_batch, Y: y_batch, F: f_batch, EPS: learning_epsilon})
            
            # Compute batch loss
            loss_val = sess.run(loss_op, feed_dict={X: x_batch, Y: y_batch, F: f_batch, EPS: learning_epsilon})
            loss_array[nidx_batch] = loss_val
        
        loss_epochs[epoch-1] = np.mean(loss_array)
        
        if epoch % display_epochs == 0 or epoch == 1:
            # Check sigma
            val_sigma_mdn = sess.run(sigma_mdn, feed_dict={X: x_batch, F: f_batch})
            minval_sigma_mdn = np.min(val_sigma_mdn)
            meanval_sigma_mdn = np.mean(val_sigma_mdn)
            maxval_sigma_mdn = np.max(val_sigma_mdn)
            
            print("epoch " + str(epoch) + ", epsilon {:.6f}".format(learning_epsilon) +
                  ", Minibatch Loss= " + "{:.7f}".format(np.mean(loss_array)) + 
                  ", sigma= " + "{:.6f}/{:.6f}/{:.6f}".format(minval_sigma_mdn,meanval_sigma_mdn,maxval_sigma_mdn))
            
        # Save
        if epoch % save_epochs == 0 or epoch == 1:
            # Now, save the graph
            filename2save1 = "./tf_saved_model/predTraj_lstmMDN_ti%dto%d" % (Hx_prev,Hx_post)
            saver.save(sess,filename2save1,global_step=100)
            
        
    print("Optimization Finished!")
    
    # Test phase
    idx_test = np.random.randint(num_traindata,size=(20,1))
    x_test = prevTraj_trainData[idx_test,h_x_train]
    x_test = np.reshape(x_test,(idx_test.shape[0],Hx_prev,dim_x))
    
    y_test = postTraj_trainData[idx_test,:]
    y_test = postTraj_trainData[:,h_y_train]
    
    f_test = f_trainData[idx_test,:]
    
    means_mdn_test = sess.run(means_mdn, feed_dict={X: x_test, F: f_test})
    sigma_mdn_test = sess.run(sigma_mdn, feed_dict={X: x_test, F: f_test})
    fracs_mdn_test = sess.run(fracs_mdn, feed_dict={X: x_test, F: f_test})
    
    # Now, save test results
    filename2save2 = "./tf_saved_model/results_predTraj_lstmMDN_ti%dto%d" % (Hx_prev,Hx_post)
    sio.savemat(filename2save2, {'means_mdn_test':means_mdn_test,'sigma_mdn_test':sigma_mdn_test,'fracs_mdn_test':fracs_mdn_test})

epoch 1, epsilon 0.000010, Minibatch Loss= -42.3842751, sigma= 0.000717/0.005304/0.029754
epoch 50, epsilon 0.000010, Minibatch Loss= -155.5826875, sigma= 0.000000/0.000411/0.008790
epoch 100, epsilon 0.000010, Minibatch Loss= -157.0464780, sigma= 0.000000/0.000300/0.009226
epoch 150, epsilon 0.000010, Minibatch Loss= -159.8415329, sigma= 0.000000/0.000622/0.023783
epoch 200, epsilon 0.000010, Minibatch Loss= -159.7974338, sigma= 0.000000/0.000713/0.035433
epoch 250, epsilon 0.000010, Minibatch Loss= -158.5976043, sigma= 0.000000/0.000444/0.013901
epoch 300, epsilon 0.000010, Minibatch Loss= -158.8930314, sigma= 0.000000/0.000419/0.053860
epoch 350, epsilon 0.000010, Minibatch Loss= -158.5232968, sigma= 0.000000/0.001003/0.085903
epoch 400, epsilon 0.000009, Minibatch Loss= -161.5180862, sigma= 0.000000/0.000878/0.037936
epoch 450, epsilon 0.000009, Minibatch Loss= -159.7439754, sigma= 0.000000/0.000537/0.062314
epoch 500, epsilon 0.000009, Minibatch Loss= -157.9385229, sigma= 0.000000

ValueError: Cannot feed value of shape (20, 1, 7) for Tensor 'Placeholder_2:0', which has shape '(?, 7)'

In [1]:
x_test

NameError: name 'x_test' is not defined